In [4]:
# 1. Imports
import pandas as pd
import numpy as np
import networkx as nx
import ahpy

# 2. Read all your pairwise‐comparison sheets from the Excel file
fn = 'comparisons.xlsx'
sheets = [
    'learning',
    'college_prep',
    'friends',
    'music_classes',
    'vocational_trng',
    'school_life'
]

# load the criteria‐vs‐criteria sheet
pairwise_criteria = pd.read_excel(fn, sheet_name='criteria', index_col=0)

# load each alternatives‐vs‐alternatives sheet
pairwise = {s: pd.read_excel(fn, sheet_name=s, index_col=0) for s in sheets}

# 3. Helper to turn an adjacency matrix into a dict of (i,j):weight
def adj_to_dict(df):
    G = nx.from_pandas_adjacency(df, create_using=nx.MultiDiGraph())
    return {
        (u, v): d['weight']
        for u, v, d in G.edges(data=True)
        if np.isfinite(d['weight'])
    }

# build comparison dicts
cmp_criteria      = adj_to_dict(pairwise_criteria)
cmp_learning      = adj_to_dict(pairwise['learning'])
cmp_college_prep  = adj_to_dict(pairwise['college_prep'])
cmp_friends       = adj_to_dict(pairwise['friends'])
cmp_music_classes = adj_to_dict(pairwise['music_classes'])
cmp_voc_trng      = adj_to_dict(pairwise['vocational_trng'])
cmp_school_life   = adj_to_dict(pairwise['school_life'])

# 4. Build AHP Compare objects
learning      = ahpy.Compare('learning',      cmp_learning,      random_index='saaty')
college_prep  = ahpy.Compare('college_prep',  cmp_college_prep,  random_index='saaty')
friends       = ahpy.Compare('friends',       cmp_friends,       random_index='saaty')
music_classes = ahpy.Compare('music_classes', cmp_music_classes, random_index='saaty')
voc_trng      = ahpy.Compare('vocational_trng',cmp_voc_trng,     random_index='saaty')
school_life   = ahpy.Compare('school_life',   cmp_school_life,   random_index='saaty')
criteria      = ahpy.Compare('criteria',      cmp_criteria,      random_index='saaty')

# wire up the hierarchy: criteria → all sub‐criteria
criteria.add_children([
    learning,
    college_prep,
    friends,
    music_classes,
    voc_trng,
    school_life
])

# 5. Print your final results
print(">> Criteria global weights:")
for k, v in criteria.global_weights.items():
    print(f"   {k:15s}: {v:.4f}")

print("\n>> Alternative (school) weights:")
for k, v in criteria.target_weights.items():
    print(f"   {k:2s}: {v:.4f}")

print("\n>> Consistency ratios (should all be < 0.1):")
for comp in [learning, friends, school_life, voc_trng, college_prep, music_classes, criteria]:
    print(f"   {comp.name:15s}: {comp.consistency_ratio:.4f}")


>> Criteria global weights:
   learning       : 0.3208
   college_prep   : 0.2374
   friends        : 0.1395
   music_classes  : 0.1391
   vocational_trng: 0.1285
   school_life    : 0.0348

>> Alternative (school) weights:
   B : 0.3782
   A : 0.3521
   C : 0.2697

>> Consistency ratios (should all be < 0.1):
   learning       : 0.0516
   friends        : 0.0000
   school_life    : 0.0000
   vocational_trng: 0.2005
   college_prep   : 0.0516
   music_classes  : 0.0825
   criteria       : 0.2272
